In [1]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.select import Select
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
import urllib.request
import time
import re

import pandas as pd
import pprint

from bs4 import BeautifulSoup

In [2]:
# 크롬 드라이버 자동 업데이트
from webdriver_manager.chrome import ChromeDriverManager

# 브라우저 꺼짐 방지
chrome_options = Options()
chrome_options.add_experimental_option("detach", True)

# 불필요한 에러 메세지 없애기
chrome_options.add_experimental_option("excludeSwitches", ["enable-logging"])
service = Service(executable_path=ChromeDriverManager().install())

In [3]:
user_agent ="Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/116.0.0.0 Safari/537.36"
chrome_options.add_argument(f'user-agent={user_agent}')

driver = webdriver.Chrome(options=chrome_options)

In [4]:
sub_urls = ['high_end_gpus.html', 'mid_range_gpus.html', 'midlow_range_gpus.html', 'low_end_gpus.html']

save_file = False  # 파일 저장 여부
waiting_sec = 2

prd_dict = {
    "name":[],
    "count":[]
    }


for sub in sub_urls:
    url = f'https://www.videocardbenchmark.net/{sub}'

    driver.get(url)
    driver.implicitly_wait(10)
    
    time.sleep(waiting_sec)

    soup = BeautifulSoup(driver.page_source, 'html.parser')
    prds = soup.select('#mark > div > div.chart_body > ul > li > a')
    for prd in prds:
        prdname = prd.select_one('a>span.prdname').get_text()
        prdcount = prd.select_one('a>span.count').get_text().replace(',','')
        
        if prdname not in prd_dict['name']:
            prd_dict['name'].append(str(prdname))
            prd_dict['count'].append(int(prdcount))
        else: continue

In [5]:
gpu_df = pd.DataFrame(prd_dict)

In [6]:
gpu_df.head()

,name,count
0,GeForce RTX 4090,38968
1,GeForce RTX 4080,34841
2,Radeon PRO W7800,32146
3,GeForce RTX 4070 Ti,31652
4,Radeon RX 7900 XTX,31173


In [7]:
len(gpu_df)

2555

In [8]:
gpu_df.to_csv("bench_mark_gpu.csv", index=None)

In [9]:
#rk4606 > a > span.count